In [ ]:
#1 Importing Relevant Libraries
import json
import string
import random 
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer 
import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout
import mysql.connector
from mysql.connector import Error
import tkinter as tk
from tkinter import scrolledtext
import time
import datetime

In [ ]:
def download_nltk_data():
    try:
        nltk.data.find('tokenizers/punkt')
    except LookupError:
        nltk.download('punkt')
    
    try:
        nltk.data.find('corpora/wordnet')
    except LookupError:
        nltk.download('wordnet')

# Download NLTK data if necessary
download_nltk_data()

In [ ]:
#2 Loading the Dataset: intents.json

with open(r'intents.json', encoding='utf-8') as file:
    data = json.load(file)

In [ ]:
#3 Extracting data_X(features) and data_Y(Target)

words = [] #For Bow model/ vocabulary for patterns
classes = [] #For Bow  model/ vocabulary for tags
data_X = [] #For storing each pattern
data_y = [] #For storing tag corresponding to each pattern in data_X 
# Iterating over all the intents

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = nltk.word_tokenize(pattern) # tokenize each pattern 
        words.extend(tokens) #and append tokens to words
        data_X.append(pattern) #appending pattern to data_X
        data_y.append(intent["tag"]) ,# appending the associated tag to each pattern 
    
    # adding the tag to the classes if it's not there already 
    if intent["tag"] not in classes:
        classes.append(intent["tag"])

# initializing lemmatizer to get stem of words
lemmatizer = WordNetLemmatizer()

# lemmatize all the words in the vocab and convert them to lowercase
# if the words don't appear in punctuation
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
# sorting the vocab and classes in alphabetical order and taking the # set to ensure no duplicates occur
words = sorted(set(words))
classes = sorted(set(classes))

In [ ]:
# 5 Text to Numbers
training = []
out_empty = [0] * len(classes)
# creating the bag of words model
for idx, doc in enumerate(data_X):
    bow = []
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)
    # mark the index of class that the current pattern is associated
    # to
    output_row = list(out_empty)
    output_row[classes.index(data_y[idx])] = 1
    # add the one hot encoded BoW and associated classes to training 
    training.append([bow, output_row])
# shuffle the data and convert it to an array
random.shuffle(training)
training = np.array(training, dtype=object)
# split the features and target labels
train_X = np.array(list(training[:, 0]))
train_Y = np.array(list(training[:, 1]))

In [ ]:
#6 The Neural Network Model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_X[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_Y[0]), activation = "softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])
model.fit(x=train_X, y=train_Y, epochs=150, verbose=1)

In [ ]:
#7 Fetching from database
def get_workout(workout_name, level):
    connection = None  # Initialize connection variable
    try:
        connection = mysql.connector.connect(
            host='localhost',
            database='workouts_db',
            user='root', 
            password='1234'
        )
        if connection.is_connected():
            cursor = connection.cursor()
            query = f"SELECT * FROM {workout_name} WHERE level = '{level}'"
            cursor.execute(query)
            results = cursor.fetchall()
            return results
    except Error as e:
        return [f"Error connecting to database: {e}"]
    finally:
        if connection is not None and connection.is_connected():
            cursor.close()
            connection.close()

In [ ]:
#8 Preprocessing the Input

def clean_text(text): 
  tokens = nltk.word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens

def bag_of_words(text, vocab): 
  tokens = clean_text(text)
  bow = [0] * len(vocab)
  for w in tokens: 
    for idx, word in enumerate(vocab):
      if word == w: 
        bow[idx] = 1
  return np.array(bow)

def pred_class(text, vocab, labels, context, intents):
  bow = bag_of_words(text, vocab)
  result = model.predict(np.array([bow]))[0]  # Extracting probabilities
  thresh = 0.5
  y_pred = [[indx, res] for indx, res in enumerate(result) if res > thresh]
  y_pred.sort(key=lambda x: x[1], reverse=True)  # Sorting by values of probability in decreasing order
  return_list = []
  
  for r in y_pred:
      for intent in intents:
          if intent['tag'] == labels[r[0]]:
              if intent['context_required'] == "" or intent['context_required'] == context:
                  return_list.append(labels[r[0]])  # Contains labels(tags) for highest probability 

  return return_list

def get_response(intents_list, intents_json, context, level, root):
  list_of_intents = intents_json["intents"]
  if not intents_list: 
      tag = 'noanswer'
  else:
      tag = intents_list[0]

  for intent in list_of_intents:
      if intent["tag"] == tag:
          if tag in ["chest_arms", "abs", "legs", "full_body", "cardio"]:
              result = random.choice(intent["responses"])
              workouts = get_workout(tag, level)
              for workout in workouts:
                id_w, exercise_name, target, repetitions, instructions, level = workout
                result += "".join(f"{exercise_name}\n *Target: {target}\n *Repetisi: {repetitions}\n *Instruksi: {instructions}\n ")
          elif tag == "level":
              level_selector = LevelSelector(root)
              root.wait_window(level_selector)  # Wait for the level selector window to close

              if level_selector.level:
                  level = level_selector.level
                  result = f"Tingkatan Anda sekarang {level}. \n"
              else:
                  result = "Anda tidak memilih tingkatan baru. \n"
          
          else:
              result = random.choice(intent["responses"])
          context = intent["context_set"]  # update context
          break
  else:
      noanswer_tag = [intent for intent in list_of_intents if intent["tag"] == "noanswer"]
      result = random.choice(noanswer_tag[0]["responses"])
      context = ""  # reset context if noanswer

  return result, tag, context, level


In [ ]:
class ChatApp(tk.Tk):
    def __init__(self, level):
        super().__init__()
        self.title("Raga Chatbot")
        self.geometry("500x600")

        # Center the window
        window_width = 500
        window_height = 600
        screen_width = self.winfo_screenwidth()
        screen_height = self.winfo_screenheight()
        x = (screen_width // 2) - (window_width // 2)
        y = (screen_height // 2) - (window_height // 2)
        self.geometry(f"{window_width}x{window_height}+{x}+{y}")

        # Set styles
        self.configure(bg='#040507')
        font = ("Helvetica", 12)
        self.transcript_font = ("Helvetica", 12)
        self.input_font = ("Helvetica", 12)
        self.button_font = ("Helvetica", 12, "bold")

        # Create chat transcript area
        self.transcript_area = scrolledtext.ScrolledText(self, wrap=tk.WORD, width=50, height=20, font=self.transcript_font, bg="#0d1117", state="disabled", fg="white")
        self.transcript_area.grid(row=0, column=0, padx=10, pady=10)

        # Create input area
        self.input_area = tk.Text(self, wrap=tk.WORD, width=50, height=3, font=self.input_font, bg="#0d1117", bd=2, relief="groove", fg="white")
        self.input_area.grid(row=1, column=0, padx=10, pady=10)

        # Create send button
        self.send_button = tk.Button(self, text="Send", font=self.button_font, bg="#4CAF50", fg="white", command=self.send_message)
        self.send_button.grid(row=2, column=0, padx=10, pady=10, sticky="e")

        # Initialize context
        self.context = ""
        self.level = level  # Set level based on user choice

    def send_message(self):
        message = self.input_area.get("1.0", tk.END).strip()
        if message:
            self.transcript_area.configure(state="normal")
            self.transcript_area.insert(tk.END, "You: " + message + "\n")
            self.transcript_area.configure(state="disabled")
            self.input_area.delete("1.0", tk.END)

            # Get response from chatbot
            intents_list = pred_class(message, words, classes, self.context, data["intents"])
            response, tagged, self.context, self.level = get_response(intents_list, data, self.context, self.level, self)
            
            self.transcript_area.configure(state="normal")
            self.transcript_area.insert(tk.END, "Raga: " + response + "\n")
            self.transcript_area.configure(state="disabled")

            if tagged == "goodbye":
                self.destroy()


In [ ]:
class LevelSelector(tk.Toplevel):
    def __init__(self, parent):
        super().__init__(parent)
        self.title("Level Selector")
        self.geometry("300x200")
        self.configure(bg='#040507')
        self.button_font = ("Helvetica", 10)
        self.label_font = ("Helvetica", 12)

        window_width = 300
        window_height = 200
        screen_width = self.winfo_screenwidth()
        screen_height = self.winfo_screenheight()
        x = (screen_width // 2) - (window_width // 2)
        y = (screen_height // 2) - (window_height // 2)
        self.geometry(f"{window_width}x{window_height}+{x}+{y}")
        
        label = tk.Label(self, font = self.label_font, fg = "white", bg = "#040507", text="Pilihlah tingkat olahraga dirimu!")
        label.pack(pady=20)
        
        beginner_button = tk.Button(self, font = self.button_font, text="Beginner", bg="#4CAF50", fg="white", command=lambda: self.select_level("beginner"))
        beginner_button.pack(pady=5)
        
        intermediate_button = tk.Button(self, font = self.button_font, text="Intermediate", bg="#4CAF50", fg="white", command=lambda: self.select_level("intermediate"))
        intermediate_button.pack(pady=5)
        
        advanced_button = tk.Button(self, font = self.button_font, text="Advanced", bg="#4CAF50", fg="white", command=lambda: self.select_level("advanced"))
        advanced_button.pack(pady=5)
        
        self.level = None
    
    def select_level(self, level):
        self.level = level
        self.destroy()

In [ ]:
if __name__ == "__main__":
    root = ChatApp(None)
    root.withdraw()  # Hide the main window

    level_selector = LevelSelector(root)
    root.wait_window(level_selector)  # Wait for the level selector window to close

    if level_selector.level:
        root.level = level_selector.level
        root.deiconify()  # Show the main window
        root.mainloop()